
<span > <h1 style="color:blue">Modelado Analitica de Texto: Arboles de desición</h1></span>

A continuación se va a presentar un modelo de clasificación usando arboles de desición de manera que se pueda predecir si una reseña podría llegar a ser positiva o negativa. Cabe resaltar que el perfilamiento y el analisis se encuentra en mayor detalle en el notebook con el nombre  Modelado_analitica_texto en la carpte Etapa 1.

## 1.Cargar librerías

In [1]:
!pip install contractions 
!pip install num2words

import nltk
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from ydata_profiling import ProfileReport

import re, string, unicodedata
from num2words import num2words
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from ydata_profiling import ProfileReport
%matplotlib qt

import contractions
import re, string, unicodedata
from num2words import num2words
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import inflect
import contractions
from langdetect import detect
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, ClassifierMixin

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.stem import LancasterStemmer, WordNetLemmatizer

C:\Users\user\AppData\Local\Temp\ipykernel_4168\1990407790.py:7: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## 2. Cargar Datos y limpieza 
La explicación de la limpiea se encuentra en el otro notebook.

In [2]:
df_datos = pd.read_excel(r'C:\Users\user\Downloads\proyecto_1_Bi_2023-2\Datos\cat_345.xlsx')
df_datos.sample(5)

,Textos_espanol,sdg
2233,El Grupo de Coordinación Interdepartamental as...,5
2658,"Los maestros, administradores y estudiantes pu...",5
1954,La financiación de la enseñanza secundaria sup...,4
569,Reconociendo que las muertes por diabetes norm...,3
2247,En la recopilación de datos en todos los ámbit...,5


In [3]:
df_datos.shape

(3000, 2)

In [4]:
df_transformacion = df_datos.copy()
df_transformacion=df_transformacion.drop_duplicates(subset=['Textos_espanol'])

In [5]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
    return new_words

def remove_specialCoders(words):
    new_words = []
    for word in words:
        if  "Ã¡" in word:
            new_word = re.sub(r'Ã¡', 'á', word)
            new_words.append(new_word)
        elif "ao" in word:
            new_word = re.sub(r'ao', 'ú', word)
            new_words.append(new_word)
        elif "Ã" in word:
            new_word = re.sub(r'Ã', 'í', word)
            new_words.append(new_word)
        elif "a3" in word:
            new_word = re.sub(r'a3', 'ó', word)
            new_words.append(new_word)
        elif "Ã©" in word:
            new_word = re.sub(r'Ã©', 'é', word)
            new_words.append(new_word)
    
        else:
            new_words.append(word)
    return new_words

def replace_numbers(words):
    """Replace all integer occurrences in a list of tokenized words with textual representation"""
    if words is None:
        return[]
    p = inflect.engine()
    new_words=[]
    for word in words:
        if word is not None: #verifica que la palabra no sea None.
            if word.isdigit():
                new_word = p.number_to_words(word)
                new_words.append(new_word)
            else:
                new_words.append(word)
    return new_words
def preprocessing(words):

    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    words = remove_specialCoders(words)
    return words

In [6]:
df_transformacion['words'] = df_transformacion['Textos_espanol'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
df_transformacion['words'] = df_transformacion['words'].apply(lambda x: ' '.join(map(str, x)))
df_transformacion['words'] = df_transformacion['words'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
df_transformacion['words'] = df_transformacion['words'].apply(lambda x: ' '.join(map(str, x)))

## 2.1. Matriz 

In [7]:
X_data, y_data = df_transformacion['words'],df_transformacion['sdg']
y_data

0       3
1       3
2       3
3       3
4       3
       ..
2995    5
2996    5
2997    5
2998    5
2999    5
Name: sdg, Length: 3000, dtype: int64

In [8]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(X_data)
print(X_dummy.shape)
X_dummy.toarray()[0]


(3000, 16852)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [9]:
nombre_caracteristica = dummy.get_feature_names_out

## 3. Modelo con Arboles de desición:

### 3.1. Variable objetivo

In [10]:
import seaborn as sns; sns.set()


In [11]:
ax = sns.countplot(x='sdg', data=df_transformacion)


### 3.2. Modelo 1:  Min samples 2 & default depth

In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X_dummy, y_data, test_size=0.2, random_state=0)
arbol = DecisionTreeClassifier(criterion='entropy', random_state=0)
tree = arbol.fit(X_train,Y_train)
y_pred = tree.predict(X_test)
matriz= confusion_matrix(Y_test, y_pred)

In [15]:
ConfusionMatrixDisplay.from_predictions(Y_test,y_pred)

In [14]:
disp= ConfusionMatrixDisplay(confusion_matrix=matriz,display_labels=arbol.classes_)
disp.plot
plt.show()

In [16]:
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           3       0.93      0.90      0.91       237
           4       0.92      0.90      0.91       187
           5       0.88      0.94      0.91       176

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600



In [17]:
from sklearn import tree

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(arbol, feature_names=dummy.get_feature_names_out(), filled=True)

In [17]:
precision = accuracy_score(Y_test, y_pred)

print("Precisión del modelo:", precision)

Precisión del modelo: 0.9116666666666666


### 3.3 Model 2: Best tree

In [18]:
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV


particiones = KFold(n_splits=10, shuffle=True, random_state = 0)
param_grid = {'criterion':['gini', 'entropy'],'max_depth':[2, 8, 10, 20, 30, 50],'min_samples_split':[2,3,4,5,6,7,8,9,10]}
tree = DecisionTreeClassifier(criterion='entropy',random_state=0)
best_model = GridSearchCV(tree, param_grid, cv=particiones)
best_model.fit(X_train, Y_train)
print(best_model.best_params_)

{'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 3}


In [19]:
best_tree = best_model.best_estimator_
y_pred_train = best_tree.predict(X_train)
y_pred_test = best_tree.predict(X_test)
matriz= confusion_matrix(Y_test, y_pred_test)
ConfusionMatrixDisplay.from_predictions(Y_test, y_pred_test)

In [20]:
print(classification_report(Y_test, y_pred_test))

              precision    recall  f1-score   support

           3       0.92      0.95      0.94       237
           4       0.95      0.91      0.93       187
           5       0.92      0.93      0.92       176

    accuracy                           0.93       600
   macro avg       0.93      0.93      0.93       600
weighted avg       0.93      0.93      0.93       600



In [21]:
from sklearn import tree

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(best_tree, feature_names=dummy.get_feature_names_out(), filled=True)

In [22]:
precision = accuracy_score(Y_test, y_pred_test)

print("Precisión del modelo:", precision)

Precisión del modelo: 0.93


### 3.4 Model 3: Tree with no parameters

In [23]:
modelo = DecisionTreeClassifier()
modelo.fit(X_train, Y_train)

DecisionTreeClassifier()

In [24]:
predicciones = modelo.predict(X_test)
precision = accuracy_score(Y_test, predicciones)
print("Precisión del modelo:", precision)

Precisión del modelo: 0.9266666666666666


In [25]:
matriz= confusion_matrix(Y_test, predicciones)
ConfusionMatrixDisplay.from_predictions(Y_test, predicciones)

In [26]:
print(classification_report(Y_test, predicciones))

              precision    recall  f1-score   support

           3       0.93      0.94      0.93       237
           4       0.95      0.91      0.93       187
           5       0.90      0.93      0.91       176

    accuracy                           0.93       600
   macro avg       0.93      0.93      0.93       600
weighted avg       0.93      0.93      0.93       600



In [27]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(modelo, feature_names=dummy.get_feature_names_out(), filled=True)

## Randomforest

In [28]:
from sklearn.datasets import load_iris
iris = load_iris()

# Model (can also use single decision tree)
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz

In [29]:
random = RandomForestClassifier(random_state=3)

In [30]:
random.fit(X_train, Y_train)

RandomForestClassifier(random_state=3)

In [31]:
predicciones = random.predict(X_test)
precision = accuracy_score(Y_test, predicciones)
print("Precisión del modelo:", precision)

Precisión del modelo: 0.9816666666666667


In [32]:
matriz= confusion_matrix(Y_test, predicciones)
ConfusionMatrixDisplay.from_predictions(Y_test, predicciones)

# Extract single tree
estimator = random.estimators_[5]

In [33]:
print(classification_report(Y_test, predicciones))

              precision    recall  f1-score   support

           3       0.99      0.97      0.98       237
           4       0.98      0.98      0.98       187
           5       0.97      0.99      0.98       176

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



In [ ]:
## imprimir arbol

In [ ]:
### 4. Conclusiones

En general se puede ver que la presión está en un valor cercano al 69%, lo cual no es muy bueno. Esto ocurre en casi todos los modelos presentados anteriormente. En base a las metricas de los diferentes modelos se puede decir que los mojores modelos van en el siguiente orden:

1. Modelo 3 con una presición del 0.699
2. Modelo 2 con una presición con el 0.694
3. Modelo 1 con una presición con un 0.691